# Packages Imports

In [1]:
import pickle
import pandas as pd
import numpy as np
import osmnx as ox
import contextily as ctx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.patches import Rectangle,Patch
from matplotlib import colors
from itertools import combinations
from math import floor, ceil
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import normalize,minmax_scale,robust_scale
import pydeck as pdk

In [2]:
PLACE_NAME = "Iskandar Puteri"

building_file = "data/iskandar_puteri.building"
amenity_file = "data/iskandar_puteri.amenity"
leisure_file = "data/iskandar_puteri.leisure"
shop_file = "data/iskandar_puteri.shop"
area_file = "data/iskandar_puteri.area"

files = [building_file,amenity_file,leisure_file,shop_file,area_file]

## reload pickle from saved file
def load_pickle_file(file):
    with open(file, "rb") as f:
        return pickle.load(f)

building,amenity,leisure,shop,area = [load_pickle_file(file) for file in files]
building = building.dropna(axis=1)
amenity = amenity.dropna(axis=1)
leisure = leisure.dropna(axis=1)
shop = shop.dropna(axis=1)
# shorthand for osmid indexes for readability reason
buil_i = building.index.get_level_values(1)
amen_i = amenity.index.get_level_values(1)
leis_i = leisure.index.get_level_values(1)
shop_i = shop.index.get_level_values(1)
# reset indexes to simplify process
building = building.reset_index()
amenity = amenity.reset_index()
leisure = leisure.reset_index()
shop = shop.reset_index()
# we found 'yes' similar in both building & shop
# Therefore, we need to change to 'building' or 'shop' instead.
building['building'].replace({'yes':'building'},inplace=True)
shop['shop'].replace({'yes':'shop'},inplace=True)
# remove 141 amenities from building
criteria = ~building['osmid'].isin(buil_i.intersection(amen_i))
building = building[criteria]
# remove 4 leisure rom building
criteria = ~building['osmid'].isin(buil_i.intersection(leis_i))
building = building[criteria]
# remove 55 shop rom building
criteria = ~building['osmid'].isin(buil_i.intersection(shop_i))
building = building[criteria]
# remove 4 shop from amenity
criteria = ~amenity['osmid'].isin(amen_i.intersection(shop_i))
amenity = amenity[criteria]
building['centroid'] = building.geometry.to_crs('EPSG:32648').centroid.to_crs('EPSG:4326')
amenity['centroid'] = amenity.geometry.to_crs('EPSG:32648').centroid.to_crs('EPSG:4326')
leisure['centroid'] = leisure.geometry.to_crs('EPSG:32648').centroid.to_crs('EPSG:4326')
shop['centroid'] = shop.geometry.to_crs('EPSG:32648').centroid.to_crs('EPSG:4326')


In [3]:
entity = pd.concat([building,amenity,leisure,shop]).reset_index(drop=True)
entity['lng'] = entity['centroid'].x
entity['lat'] = entity['centroid'].y
df = entity[['lng','lat']]

In [4]:
BLUES = [
        [168, 221, 181],[123, 204, 196],[67, 162, 202],[8, 104, 172],
        ]
GREENS = [
        (153,216,201),(102,194,164),(44,162,95),(0,109,44)
        ]
PINKS = [
        (201,148,199),(223,101,176),(221,28,119),(152,0,67)
        ]
REDS = [
        (254,178,76),(253,141,60),(240,59,32),(189,0,38)
        ]
RED2BLUE = ((5,113,176),(146,197,222),(244,165,130),(202,0,32))


In [5]:
view = pdk.data_utils.compute_view(df[['lng','lat']])
view.pitch = 50
view.bearing = -20
view.zoom = 11
view.latitude = 1.436019010084092
view.longitude = 103.61427530063695
view
INITIAL_VIEW_STATE = view

layer = pdk.Layer(
  'GridLayer',
  data=df,
  get_position=['lng','lat'],
  cell_size=400,
  opacity=0.6,
  colorRange=RED2BLUE,
  extruded=True,
  elevation_scale=3
)
pdk.Deck(
  layers=[layer],
  initial_view_state=INITIAL_VIEW_STATE,
  map_style=pdk.map_styles.CARTO_ROAD,
  height=700,
  width=700,
).show()


DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, height=700, json_input='{\n  "initialV…

In [6]:
view = pdk.data_utils.compute_view(df[['lng','lat']])
view.pitch = 50
view.bearing = -20
view.zoom = 11
view.latitude = 1.436019010084092
view.longitude = 103.61427530063695
INITIAL_VIEW_STATE = view

layer = pdk.Layer(
  'HexagonLayer',
  data=df,
  get_position=['lng','lat'],
  radius=200,
  opacity=0.6,
  colorRange=RED2BLUE,
  extruded=True,
  elevation_scale=3
)
pdk.Deck(
  layers=[layer],
  initial_view_state=INITIAL_VIEW_STATE,
  map_style=pdk.map_styles.CARTO_DARK,
  height=700,
  width=700,
).show()

DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, height=700, json_input='{\n  "initialV…

In [7]:
view = pdk.data_utils.compute_view(df[['lng','lat']])
view.pitch = 50
view.bearing = -20
view.zoom = 11
view.latitude = 1.436019010084092
view.longitude = 103.61427530063695
INITIAL_VIEW_STATE = view

layer = pdk.Layer(
  'HeatmapLayer',
  data=df,
  get_position=['lng','lat'],
  radius=200,
  opacity=0.6,
  colorRange=RED2BLUE,
  radius_pixels = 70
)
pdk.Deck(
  layers=[layer],
  initial_view_state=INITIAL_VIEW_STATE,
  map_style=pdk.map_styles.CARTO_LIGHT,
  height=700,
  width=700,
).show()

DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, height=700, json_input='{\n  "initialV…